In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [4]:
import json
import pickle
from copy import deepcopy
from pathlib import Path

import gurobipy
from gurobipy import GRB
import dgl
import numpy as np
import torch
import matplotlib.pyplot as plt

from src.problem import get_model
from src.dataset import InstanceDataset
from src.net import InstanceGCN
from src.utils import load_from_wandb

In [8]:
instances_dir = Path('../data/raw')
instances_fpaths = list(instances_dir.glob('97_*.json'))

net = InstanceGCN(1, readout_op=None)
net = load_from_wandb(net, 'vty6z1q3', 'sat-gnn', 'model_last')
net.eval()

ds = InstanceDataset(instances_fpaths, split='val', return_model=True)

g, _, m = ds[0]
m

<gurobi.Model MIP instance Unnamed: 5913 constrs, 2037 vars, Parameter changes: TimeLimit=60.0, Username=(user-defined), LogToConsole=0>

In [33]:
def get_ef_times_n_vars(graph, model, net, name=None, plot=True, timeout=60):
    vars_names = np.core.defchararray.array([v.getAttr(GRB.Attr.VarName) for v in model.getVars()])
    vars_names = vars_names[(vars_names.find('x(') >= 0) | (vars_names.find('phi(') >= 0)]

    # baseline results
    model_ = model.copy()
    model_.setParam('TimeLimit', timeout)
    model_.update()
    model_.optimize()
    baseline_runtime = model_.Runtime
    baseline_obj = model_.ObjVal
    baseline_gap = model_.MIPGap

    with torch.no_grad():
        x_hat = torch.sigmoid(net(graph)).squeeze(0)

    most_certain_idx  = (x_hat - 0.5).abs().sort(descending=True).indices


    runtimes = list()
    objs = list()
    gaps = list()
    ns = list()
    for n in [0, 50, 100, 150, 200, 300, 500, 1000, len(x_hat)]:
    # for n in [0, 50, 100, 150]:
        if n == 0:
            runtimes.append(baseline_runtime)
            objs.append(baseline_obj)
            gaps.append(baseline_gap)
            ns.append(n)
            continue

        fixed_x_hat = (x_hat[most_certain_idx[:n]] > .5).to(x_hat)
        fixed_vars_names = vars_names[most_certain_idx[:n]]

        # fix variables
        model_ = model.copy()
        for fixed_var_name, fixed_var_X in zip(fixed_vars_names, fixed_x_hat):
            model_.getVarByName(fixed_var_name).lb = fixed_var_X
            model_.getVarByName(fixed_var_name).ub = fixed_var_X

        model_.setParam('TimeLimit', timeout)
        model_.update()
        model_.optimize()

        if model_.status not in [2, 9]:
            print('early fixing with n=',n,' made the optimizatio terminate with status ',model_.status)
            break

        runtimes.append(model_.Runtime)
        objs.append(model_.ObjVal)
        gaps.append(model_.MIPGap)
        ns.append(n)

    objs = [100 * o / max(objs) for o in objs]

    if plot:
        fig, ax = plt.subplots()
        ax.plot(ns, runtimes, '.-', label='Runtime')
        # ax.set_xlim((0, x_opt.shape[0]))
        ax.grid()
        ax.legend(loc=1)

        ax.set_title('Early fixing performance '+str(name))
        ax.set_xlabel('# fixed variables')
        ax.set_ylabel('Runtime')

        ax.set_xlim(0,max(ns))

        obj_ax = ax.twinx()
        obj_ax.plot(ns, objs, '.-', c='r', label='Objective')
        obj_ax.legend(loc=4)
        obj_ax.set_ylim(min(objs), 100)
        obj_ax.set_ylabel('Objective')

        fig.show()

    return ns, runtimes, objs, gaps

In [43]:
len(js), len(ts)

(9, 97)

In [41]:
x_vars = [v.varname for v in m.getVars() if 'x(' in v.varname]
js = {int(var[len('x('):].split(',')[0]) for var in x_vars}
ts = {int(var.split(',')[1][:-1]) for var in x_vars}

In [35]:
g, _, m = ds[2]

ns, runtimes, objs, gaps = get_ef_times_n_vars(g, m, net, timeout=10, plot=False)
ns, runtimes, objs, gaps

early fixing with n= 500  made the optimizatio terminate with status  3


([0, 50, 100, 150, 200, 300],
 [10.012699842453003,
  10.0067880153656,
  10.004565954208374,
  10.008992910385132,
  10.008025884628296,
  10.010357141494751],
 [100.0, 100.0, 100.0, 100.0, 100.0, 100.0],
 [0.007909948281107393,
  0.004867660480681472,
  0.0027380590203833284,
  0.000608457560085184,
  0.000304228780042592,
  0.000304228780042592])